In [4]:
import os
from PIL import Image

def resize_images(input_folder, output_folder, size):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        input_image_path = os.path.join(input_folder, filename)
        # Check if the file is an image
        if os.path.isfile(input_image_path) and any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.gif']):
            # Open the image
            original_image = Image.open(input_image_path)
            # Resize the image
            resized_image = original_image.resize((size, size), Image.ANTIALIAS)
            # Save the resized image to the output folder
            output_image_path = os.path.join(output_folder, filename)
            resized_image.save(output_image_path)

# Example usage:
input_folder = "stripe_rust"
output_folder = "new_stripe_rust"
desired_size = 1500

resize_images(input_folder, output_folder, desired_size)


C:\Users\nrgaj\AppData\Local\Temp\ipykernel_30756\558325107.py:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = original_image.resize((size, size), Image.ANTIALIAS)


In [3]:
from PIL import Image

# Open the image
image = Image.open("new_septoria/los(12).JPG")

# Get the image shape (width, height)
image_shape = image.size

# Display the shape
print("Image shape:", image_shape)

Image shape: (1500, 1500)


In [6]:
from PIL import Image

def partition_image(input_image_path, output_folder):
    # Open the image
    image = Image.open(input_image_path)
    width, height = image.size
    
    # Check if image can be partitioned into 3x3 grid
    if width % 500 != 0 or height % 500 != 0:
        print("Image dimensions not divisible by 500. Cannot partition.")
        return
    
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    k=1
    # Partition the image
    for i in range(3):
        for j in range(3):
            # Define the region to crop
            left = j * 500
            upper = i * 500
            right = left + 500
            lower = upper + 500
            
            # Crop the image
            cropped_image = image.crop((left, upper, right, lower))
            
            # Save the cropped image
            output_image_path = os.path.join(output_folder, f"partiton_{k}.jpg")
            cropped_image.save(output_image_path)
            k=k+1

# Example usage
input_image_path = "new_healthy/loh(1).JPG"
output_folder = "output_images"

partition_image(input_image_path, output_folder)


In [11]:
import os
from PIL import Image

def partition_images(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        input_image_path = os.path.join(input_folder, filename)
        # Check if the file is an image
        if os.path.isfile(input_image_path) and any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.gif']):
            partition_image(input_image_path, output_folder)

def partition_image(input_image_path, output_folder):
    # Open the image
    image = Image.open(input_image_path)
    width, height = image.size
    
    # Check if image can be partitioned into 3x3 grid
    if width % 500 != 0 or height % 500 != 0:
        print(f"Image {input_image_path} dimensions not divisible by 500. Cannot partition.")
        return
    k=1
    # Partition the image
    for i in range(3):
        for j in range(3):
            # Define the region to crop
            left = j * 500
            upper = i * 500
            right = left + 500
            lower = upper + 500
            
            # Crop the image
            cropped_image = image.crop((left, upper, right, lower))
            
            # Save the cropped image
            output_image_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(input_image_path))[0]}_partition{k}.jpg")
            cropped_image.save(output_image_path)
            k=k+1

# Example usage
input_folder = "new_septoria"
output_folder = "partition_septoria"

partition_images(input_folder, output_folder)


In [24]:
import os

# Function to create a text file with image paths and labels
def create_label_file(input_folder, output_file, label):
    # Open the output file in write mode
    with open(output_file, 'a') as f:
        # Iterate over the files in the input folder
        for filename in os.listdir(input_folder):
            # Check if the file is an image
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                # Write the image path and label to the file separated by a comma
                image_path = os.path.join(input_folder, filename)
                f.write(image_path + ',' + str(label) + '\n')

# Example usage
input_folder = "aug_stripe_rust"
output_file = "labels.txt"
label = 0

# Create the label file
create_label_file(input_folder, output_file, label)


In [1]:
import random

def split_text_file(input_file, output_file_1, output_file_2, split_ratio=0.8):
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    # Shuffle the lines randomly
    random.shuffle(lines)
    
    total_lines = len(lines)
    split_index = int(total_lines * split_ratio)
    
    # Split the lines
    part1 = lines[:split_index]
    part2 = lines[split_index:]
    
    # Write to output files
    with open(output_file_1, 'w') as f:
        f.writelines(part1)
    
    with open(output_file_2, 'w') as f:
        f.writelines(part2)

# Example usage:
input_file = "labels.txt"  # Path to your input text file
output_file_1 = "train.txt"  # Path to the first output file (80%)
output_file_2 = "test.txt"  # Path to the second output file (20%)
split_ratio = 0.8  # 80% for the first file, 20% for the second file

split_text_file(input_file, output_file_1, output_file_2, split_ratio)


In [18]:
import os
import Augmentor

def perform_augmentation(input_folder, output_folder, num_augmentations_per_image):
    # Initialize an Augmentor Pipeline
    p = Augmentor.Pipeline(input_folder, output_folder)

    # Define augmentation operations
    p.rotate(probability=0.25, max_left_rotation=10, max_right_rotation=10)
    p.rotate(probability=0.25, max_left_rotation=20, max_right_rotation=20)
    p.rotate(probability=0.25, max_left_rotation=15, max_right_rotation=15)
    p.rotate(probability=0.25, max_left_rotation=25, max_right_rotation=25)
    p.flip_left_right(probability=0.5)
    p.flip_top_bottom(probability=0.5)

    # Perform augmentation for each image in the input folder
    p.sample(num_augmentations_per_image)

# Example usage
input_folder = "new_healthy"
output_folder = "augmented_images"
num_original_images = 1
num_augmentations_per_image = 8

# Perform augmentation for original images
perform_augmentation(input_folder, output_folder, num_augmentations_per_image * num_original_images)

Initialised with 102 image(s) found.
Output directory set to new_healthy\augmented_images.

Processing <PIL.Image.Image image mode=RGB size=1500x1500 at 0x29942D77B80>: 100%|█| 8/8 [00:00<00:00, 11.00 Samples/s]


In [16]:
!pip install Augmentor


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import os
import random
import requests

# List of synset IDs for ImageNet categories
synset_ids = ['n02123597', 'n04037443', 'n04584207']  # Example synset IDs

# Number of images to download per category
num_images_per_category = 5

# Output directory to save downloaded images
output_dir = 'imagenet_images'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over synset IDs
for synset_id in synset_ids:
    # Download random images from ImageNet
    for i in range(num_images_per_category):
        # Generate random image ID
        image_id = random.randint(1, 1000)
        
        # Image URL
        image_url = f"http://www.image-net.org/api/text/imagenet.synset.geturls?wnid={synset_id}"
        
        try:
            # Download image
            response = requests.get(image_url, timeout=10)
            image_urls = response.text.split('\r\n')
            random_image_url = random.choice(image_urls)
            image_data = requests.get(random_image_url, timeout=10).content
            
            # Save image
            with open(os.path.join(output_dir, f"{synset_id}_{image_id}.jpg"), 'wb') as f:
                f.write(image_data)
        except Exception as e:
            print(f"Error downloading image: {e}")


Error downloading image: HTTPConnectionPool(host='www.image-net.org', port=80): Max retries exceeded with url: /api/text/imagenet.synset.geturls?wnid=n02123597 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000029943172590>, 'Connection to www.image-net.org timed out. (connect timeout=10)'))
Error downloading image: HTTPConnectionPool(host='www.image-net.org', port=80): Max retries exceeded with url: /api/text/imagenet.synset.geturls?wnid=n02123597 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000029943172CE0>, 'Connection to www.image-net.org timed out. (connect timeout=10)'))
Error downloading image: HTTPConnectionPool(host='www.image-net.org', port=80): Max retries exceeded with url: /api/text/imagenet.synset.geturls?wnid=n02123597 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000299422BDA50>, 'Connection to www.image-net.org timed out. (connect timeout=10)'))
Error downloading image: 

KeyboardInterrupt: 

In [21]:
import os
import random
import shutil

# Define the directory containing subfolders with images
base_dir = 'C:\naimish\imageNet\imagenet-mini\train'

# Define the directory to save selected images
output_dir = 'non_leaf'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the number of images to select
num_images = 3663

# List all subdirectories
subfolders = [os.path.join(base_dir, name) for name in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, name))]

# Initialize a list to store selected image paths
selected_images = []

# Iterate through subfolders and select random images
for folder in subfolders:
    images = [os.path.join(folder, img) for img in os.listdir(folder) if os.path.isfile(os.path.join(folder, img))]
    selected_images.extend(random.sample(images, min(num_images // len(subfolders), len(images))))

# Copy selected images to the output directory
for img_path in selected_images:
    shutil.copy(img_path, output_dir)


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'C:\naimish\\imageNet\\imagenet-mini\train'